In [1]:
from sched import scheduler
%cd ../..

/Users/aflamant/Documents/courses/2024-2025/mémoire/03-code/memoire/MLP


# Import libraries

In [4]:
RANDOM_STATE = 42
import numpy as np

import os
import torch
from torch import nn
from torch.utils.data import random_split, DataLoader, Subset
from sklearn.model_selection import KFold
from dataset import (TenBarsCantileverTrussSingleEADataset,
                     TwoBarsTrussSingleEADataset,
                     BiSupportedTrussBeamSingleEADataset)

import mlflow
from mlflow.models import infer_signature

from models.architecture import MultiLayerPerceptron
from models.processing import StandardScaler

np.random.seed(RANDOM_STATE)

# Data

In [25]:
names = ['beam', 'cantilever', 'triangle']

path = {
    'beam': 'data/dataset/beam/data.hdf5',
    'cantilever': 'data/dataset/cantilever/data.hdf5',
    'triangle': 'data/dataset/triangle/data.hdf5'
}

_dataset = {
    'beam': BiSupportedTrussBeamSingleEADataset(path['beam']),
    'cantilever': TenBarsCantileverTrussSingleEADataset(path['cantilever']),
    'triangle': TwoBarsTrussSingleEADataset(path['triangle'])
}

In [5]:
_train_ds = {}
_test_ds = {}

for typology in names:
    train_ds, test_ds = random_split(_dataset[typology], (0.8, 0.2))
    _train_ds[typology] = train_ds
    _test_ds[typology] = test_ds

# Model

In [6]:
device = torch.device(
    'cuda' if torch.cuda.is_available()
    else 'mps' if torch.backends.mps.is_available()
    else 'cpu'
)

In [7]:
def train_model(model, train_loader, criterion, optimizer, x_scaler, y_scaler, device):
    model.train()
    total_loss = 0.0
    for batch in train_loader:
        inputs, targets, _, _, _ = batch
        inputs, targets = inputs.to(device), targets.to(device)

        inputs = x_scaler.transform(inputs)
        targets = y_scaler.transform(targets)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(train_loader)

In [8]:
# Validation loop
def validate_model(model, val_loader, criterion, x_scaler, y_scaler, device):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            inputs, targets, _, _, _ = batch
            inputs, targets = inputs.to(device), targets.to(device)

            inputs = x_scaler.transform(inputs)
            targets = y_scaler.transform(targets)

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    return total_loss / len(val_loader)

In [9]:
def main(
        tb_writer,
        dataset,
        out_size=1,
        hidden_size=70,
        n_hidden=3,
        activation=nn.LeakyReLU,
        activation_params={'negative_slope': 0.01},
        learning_rate=1e-3,
        n_folds=3,
        n_epochs=30,
):
    in_size = len(dataset[0][0])

    train_losses_fold = [[] for _ in range(n_folds)]
    val_losses_fold = [[] for _ in range(n_folds)]

    kf = KFold(n_splits=n_folds, shuffle=True, random_state=RANDOM_STATE)
    for fold, (train_indices, test_indices) in enumerate(kf.split(dataset)):
        N_EPOCHS = n_epochs
        BATCH_SIZE = 2048  # Not an hyperparameter
        LEARNING_RATE = learning_rate

        train_ds = Subset(dataset, train_indices)
        test_ds = Subset(dataset, test_indices)

        train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, )
        val_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, )

        x_scaler = StandardScaler(in_size).to(device)
        y_scaler = StandardScaler(out_size).to(device)
        for x, y, _, _, _ in train_loader:
            x_scaler.partial_fit(x.to(device))
            y_scaler.partial_fit(y.to(device))

        model = MultiLayerPerceptron(in_size, out_size, hidden_size, n_hidden, activation,
                                     activation_params).to(device)

        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

        for epoch in range(N_EPOCHS):
            train_loss = train_model(model, train_loader, criterion, optimizer, x_scaler, y_scaler, device)
            val_loss = validate_model(model, val_loader, criterion, x_scaler, y_scaler, device)

            train_losses_fold[fold].append(train_loss)
            val_losses_fold[fold].append(val_loss)

            tb_writer.add_scalar(f'Loss/train_FOLD_{fold}', train_loss, epoch)
            tb_writer.add_scalar(f'Loss/val_FOLD_{fold}', val_loss, epoch)

            print(
                f"Fold {fold + 1}, Epoch {epoch + 1}/{N_EPOCHS}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        print(f"Finished fold {fold + 1}")
    train_losses = np.mean(train_losses_fold, axis=0)
    val_losses = np.mean(val_losses_fold, axis=0)

    for i in range(len(train_losses)):
        tb_writer.add_scalar(f'Loss/train', train_losses[i], i)
        tb_writer.add_scalar(f'Loss/val', val_losses[i], i)


In [22]:
activation = nn.LeakyReLU
activation_params = {'negative_slope': 0.05}
lr = 1e-3
dataset_name = 'cantilever'
for n_layers in [2, 3, 4, 5]:
    for layer_size in [30, 40, 50, 60, 70]:
        if len(activation_params) == 0:
            writer = SummaryWriter(f"./runs/{dataset_name}/MLP/{n_layers}/{layer_size}/{activation.__name__}/lr_{lr}")
        else:
            writer = SummaryWriter(
                f"./runs/{dataset_name}/MLP/{n_layers}/{layer_size}/{activation.__name__}/{"".join([f"{k}_{v}" for k, v in activation_params.items()])}/lr_{lr}")

        main(writer, _train_ds[dataset_name], hidden_size=layer_size, n_hidden=n_layers, learning_rate=lr,
             activation=activation, activation_params=activation_params)

Fold 1, Epoch 1/30, Train Loss: 0.8754, Val Loss: 0.7422
Fold 1, Epoch 2/30, Train Loss: 0.5947, Val Loss: 0.4179
Fold 1, Epoch 3/30, Train Loss: 0.3277, Val Loss: 0.2674
Fold 1, Epoch 4/30, Train Loss: 0.2333, Val Loss: 0.2036
Fold 1, Epoch 5/30, Train Loss: 0.1813, Val Loss: 0.1636
Fold 1, Epoch 6/30, Train Loss: 0.1482, Val Loss: 0.1380
Fold 1, Epoch 7/30, Train Loss: 0.1267, Val Loss: 0.1210
Fold 1, Epoch 8/30, Train Loss: 0.1117, Val Loss: 0.1084
Fold 1, Epoch 9/30, Train Loss: 0.1003, Val Loss: 0.0980
Fold 1, Epoch 10/30, Train Loss: 0.0911, Val Loss: 0.0898
Fold 1, Epoch 11/30, Train Loss: 0.0835, Val Loss: 0.0827
Fold 1, Epoch 12/30, Train Loss: 0.0769, Val Loss: 0.0764
Fold 1, Epoch 13/30, Train Loss: 0.0714, Val Loss: 0.0713
Fold 1, Epoch 14/30, Train Loss: 0.0667, Val Loss: 0.0667
Fold 1, Epoch 15/30, Train Loss: 0.0627, Val Loss: 0.0628
Fold 1, Epoch 16/30, Train Loss: 0.0592, Val Loss: 0.0590
Fold 1, Epoch 17/30, Train Loss: 0.0558, Val Loss: 0.0558
Fold 1, Epoch 18/30, Tr

In [23]:
# for triangle best is 3x70
# for beam best is 3x60
# for cantilever best is 4x60 -> But 4x40 is great
n_layers = 4
layer_size = 40
activation = nn.LeakyReLU
activation_params = {'negative_slope': 0.05}
for lr in np.logspace(-4, -1, num=6, ):
    if len(activation_params) == 0:
        writer = SummaryWriter(f"./runs/{dataset_name}/MLP/{n_layers}/{layer_size}/{activation.__name__}/lr_{lr}")
    else:
        writer = SummaryWriter(
            f"./runs/{dataset_name}/MLP/{n_layers}/{layer_size}/{activation.__name__}/{"".join([f"{k}_{v}" for k, v in activation_params.items()])}/lr_{lr}")

    main(writer, _train_ds[dataset_name], hidden_size=layer_size, n_hidden=n_layers, learning_rate=lr,
         activation=activation, activation_params=activation_params, n_epochs=100)

Fold 1, Epoch 1/100, Train Loss: 0.9980, Val Loss: 0.9892
Fold 1, Epoch 2/100, Train Loss: 0.9812, Val Loss: 0.9675
Fold 1, Epoch 3/100, Train Loss: 0.9507, Val Loss: 0.9230
Fold 1, Epoch 4/100, Train Loss: 0.8941, Val Loss: 0.8508
Fold 1, Epoch 5/100, Train Loss: 0.8135, Val Loss: 0.7608
Fold 1, Epoch 6/100, Train Loss: 0.7144, Val Loss: 0.6476
Fold 1, Epoch 7/100, Train Loss: 0.5818, Val Loss: 0.5009
Fold 1, Epoch 8/100, Train Loss: 0.4340, Val Loss: 0.3676
Fold 1, Epoch 9/100, Train Loss: 0.3268, Val Loss: 0.2905
Fold 1, Epoch 10/100, Train Loss: 0.2657, Val Loss: 0.2445
Fold 1, Epoch 11/100, Train Loss: 0.2274, Val Loss: 0.2139
Fold 1, Epoch 12/100, Train Loss: 0.2011, Val Loss: 0.1924
Fold 1, Epoch 13/100, Train Loss: 0.1818, Val Loss: 0.1759
Fold 1, Epoch 14/100, Train Loss: 0.1666, Val Loss: 0.1623
Fold 1, Epoch 15/100, Train Loss: 0.1539, Val Loss: 0.1507
Fold 1, Epoch 16/100, Train Loss: 0.1429, Val Loss: 0.1405
Fold 1, Epoch 17/100, Train Loss: 0.1334, Val Loss: 0.1319
Fold 1

In [24]:
# For triangle best is 3x70 ReLU lr 4e-4
# For beam 3x60 ReLU lr 4e-4
# For cantilever 4x40 ReLU lr 4e-4
n_layers = 4
layer_size = 40
lr = 4e-4

for activation, activation_params in zip(
        [nn.ReLU, nn.Tanh, nn.Sigmoid, nn.LeakyReLU, nn.LeakyReLU, nn.LeakyReLU],
        [{}, {}, {}, {'negative_slope': 0.01}, {'negative_slope': 0.02}, {'negative_slope': 0.05}]
):
    if len(activation_params) == 0:
        writer = SummaryWriter(f"./runs/{dataset_name}/MLP/{n_layers}/{layer_size}/{activation.__name__}/lr_{lr}")
    else:
        writer = SummaryWriter(
            f"./runs/{dataset_name}/MLP/{n_layers}/{layer_size}/{activation.__name__}/{"".join([f"{k}_{v}" for k, v in activation_params.items()])}/lr_{lr}")

    main(writer, _train_ds[dataset_name], hidden_size=layer_size, n_hidden=n_layers, learning_rate=lr,
         activation=activation, activation_params=activation_params, n_epochs=50)

Fold 1, Epoch 1/50, Train Loss: 0.9698, Val Loss: 0.9121
Fold 1, Epoch 2/50, Train Loss: 0.7999, Val Loss: 0.6349
Fold 1, Epoch 3/50, Train Loss: 0.4544, Val Loss: 0.3158
Fold 1, Epoch 4/50, Train Loss: 0.2541, Val Loss: 0.2030
Fold 1, Epoch 5/50, Train Loss: 0.1708, Val Loss: 0.1469
Fold 1, Epoch 6/50, Train Loss: 0.1302, Val Loss: 0.1210
Fold 1, Epoch 7/50, Train Loss: 0.1105, Val Loss: 0.1054
Fold 1, Epoch 8/50, Train Loss: 0.0969, Val Loss: 0.0934
Fold 1, Epoch 9/50, Train Loss: 0.0861, Val Loss: 0.0836
Fold 1, Epoch 10/50, Train Loss: 0.0773, Val Loss: 0.0770
Fold 1, Epoch 11/50, Train Loss: 0.0705, Val Loss: 0.0722
Fold 1, Epoch 12/50, Train Loss: 0.0649, Val Loss: 0.0652
Fold 1, Epoch 13/50, Train Loss: 0.0604, Val Loss: 0.0630
Fold 1, Epoch 14/50, Train Loss: 0.0567, Val Loss: 0.0593
Fold 1, Epoch 15/50, Train Loss: 0.0532, Val Loss: 0.0529
Fold 1, Epoch 16/50, Train Loss: 0.0500, Val Loss: 0.0534
Fold 1, Epoch 17/50, Train Loss: 0.0473, Val Loss: 0.0499
Fold 1, Epoch 18/50, Tr

For beam
3x60
lr = 0.0004
ReLU

In [21]:
# for triangle best is 3x70 ReLU lr 4e-4

Final training

In [19]:
def main_final(
        tb_writer,
        train_dataset,
        test_dataset,
        out_size=1,
        hidden_size=70,
        n_hidden=3,
        activation=nn.ReLU,
        activation_params={},
        learning_rate=1e-4,
        n_epochs=30,
        save_path = None
):
    in_size = len(train_dataset[0][0])

    N_EPOCHS = n_epochs
    BATCH_SIZE = 2048  # Not an hyperparameter
    LEARNING_RATE = learning_rate

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, )
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, )

    x_scaler = StandardScaler(in_size).to(device)
    y_scaler = StandardScaler(out_size).to(device)

    for x, y, _, _, _ in train_loader:
        x_scaler.partial_fit(x.to(device))
        y_scaler.partial_fit(y.to(device))

    model = MultiLayerPerceptron(in_size, out_size, hidden_size, n_hidden, activation,
                                 activation_params).to(device)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5, verbose=True)
    best_test_loss = float('inf')
    for epoch in range(N_EPOCHS):
        train_loss = train_model(model, train_loader, criterion, optimizer, x_scaler, y_scaler, device)

        model.eval()
        total_test_loss = 0.0
        total_MAE = 0.0
        total_MAER = 0.0
        total_MSE = 0.0
        with torch.no_grad():
            for batch in test_loader:
                inputs, targets, _, _, _ = batch
                inputs, targets = inputs.to(device), targets.to(device)

                inputs = x_scaler.transform(inputs)
                targets_scaled = y_scaler.transform(targets)

                outputs = model(inputs)
                loss = criterion(outputs, targets_scaled)
                total_test_loss += loss.item()

                outputs_unscaled = y_scaler.inverse_transform(outputs)
                loss = criterion(outputs_unscaled, targets)
                total_MSE += loss.item()

                loss = torch.abs(targets - outputs_unscaled).mean()
                total_MAE += loss.item()

                loss = (torch.abs(targets - outputs_unscaled)/targets).mean()
                total_MAER += loss.item()


        test_loss =  total_test_loss / len(test_loader)
        test_MAE = total_MAE / len(test_loader)
        test_MAER = total_MAER / len(test_loader)
        test_MSE = total_MSE / len(test_loader)

        scheduler.step(train_loss)

        tb_writer.add_scalar(f'Loss/train', train_loss, epoch)
        tb_writer.add_scalar(f'Loss/test', test_loss, epoch)
        tb_writer.add_scalar(f'MAE/test', test_MAE, epoch)
        tb_writer.add_scalar(f'MAER/test', test_MAER, epoch)
        tb_writer.add_scalar(f'MSE/test', test_MSE, epoch)

        print(f"Epoch {epoch + 1}/{N_EPOCHS}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test MAER: {test_MAER*100:.4f} %, Test MAE: {test_MAE*1e-6:.4f} MN, Test MSE: {test_MSE*1e-12:.4f} MN^2", end=' ')

        if test_loss < best_test_loss and save_path is not None and epoch > 50:
            best_test_loss = test_loss
            torch.save(model.state_dict(), f"{save_path}/best_model_{test_loss}.pth")
            print("-> MODEL SAVED")
        else:
            print("")

    return model

Epoch 1/200, Train Loss: 0.8600, Test Loss: 0.6333, Test MAER: 63.7375 %, Test MAE: 335.4911 MN, Test MSE: 163957.2338 MN^2 
Epoch 2/200, Train Loss: 0.3019, Test Loss: 0.1424, Test MAER: 20.5246 %, Test MAE: 122.6851 MN, Test MSE: 36865.3345 MN^2 
Epoch 3/200, Train Loss: 0.1145, Test Loss: 0.1025, Test MAER: 16.3868 %, Test MAE: 99.1628 MN, Test MSE: 26538.3581 MN^2 
Epoch 4/200, Train Loss: 0.0890, Test Loss: 0.0835, Test MAER: 14.0114 %, Test MAE: 84.8277 MN, Test MSE: 21624.1061 MN^2 
Epoch 5/200, Train Loss: 0.0734, Test Loss: 0.0696, Test MAER: 12.3345 %, Test MAE: 73.9297 MN, Test MSE: 18012.2825 MN^2 
Epoch 6/200, Train Loss: 0.0622, Test Loss: 0.0601, Test MAER: 11.0158 %, Test MAE: 65.8705 MN, Test MSE: 15564.2313 MN^2 
Epoch 7/200, Train Loss: 0.0543, Test Loss: 0.0541, Test MAER: 10.3993 %, Test MAE: 61.4593 MN, Test MSE: 14004.1472 MN^2 
Epoch 8/200, Train Loss: 0.0481, Test Loss: 0.0486, Test MAER: 9.3464 %, Test MAE: 55.5729 MN, Test MSE: 12578.2893 MN^2 
Epoch 9/200, T

RuntimeError: Parent directory ./runs/final/beam/MLP/3/60/ReLU/lr_0.0004/SAVE does not exist.

In [22]:
writer = SummaryWriter(f"./runs/final/beam/MLP/3/60/ReLU/lr_0.0004")
save_path = "./runs/final/beam/MLP/3/60/ReLU/lr_0.0004/SAVE"
os.makedirs(save_path, exist_ok=True)
model_beam = main_final(writer, _train_ds['beam'], _test_ds['beam'], hidden_size=60, n_hidden=3, activation=nn.ReLU, activation_params={}, learning_rate=0.0004, n_epochs=200, save_path=save_path)
writer.close()

/Users/aflamant/miniconda3/envs/memoire/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/200, Train Loss: 0.8711, Test Loss: 0.6568, Test MAER: 66.1410 %, Test MAE: 342.9086 MN, Test MSE: 170058.7456 MN^2 
Epoch 2/200, Train Loss: 0.3420, Test Loss: 0.1633, Test MAER: 23.1527 %, Test MAE: 136.5396 MN, Test MSE: 42286.7302 MN^2 
Epoch 3/200, Train Loss: 0.1322, Test Loss: 0.1174, Test MAER: 18.1150 %, Test MAE: 108.8518 MN, Test MSE: 30389.8478 MN^2 
Epoch 4/200, Train Loss: 0.1009, Test Loss: 0.0941, Test MAER: 15.2288 %, Test MAE: 92.2502 MN, Test MSE: 24368.7492 MN^2 
Epoch 5/200, Train Loss: 0.0826, Test Loss: 0.0799, Test MAER: 13.7805 %, Test MAE: 82.7988 MN, Test MSE: 20677.8948 MN^2 
Epoch 6/200, Train Loss: 0.0700, Test Loss: 0.0670, Test MAER: 12.1723 %, Test MAE: 72.4980 MN, Test MSE: 17347.9364 MN^2 
Epoch 7/200, Train Loss: 0.0607, Test Loss: 0.0597, Test MAER: 11.4738 %, Test MAE: 67.5908 MN, Test MSE: 15445.6603 MN^2 
Epoch 8/200, Train Loss: 0.0534, Test Loss: 0.0529, Test MAER: 10.0205 %, Test MAE: 60.1464 MN, Test MSE: 13688.6118 MN^2 
Epoch 9/200,

In [23]:
writer = SummaryWriter(f"./runs/final/triangle/MLP/3/70/ReLU/lr_0.0004")
save_path = "./runs/final/triangle/MLP/3/70/ReLU/lr_0.0004/SAVE"
os.makedirs(save_path, exist_ok=True)
model_beam = main_final(writer, _train_ds['triangle'], _test_ds['triangle'], hidden_size=70, n_hidden=3, activation=nn.ReLU, activation_params={}, learning_rate=0.0004, n_epochs=200, save_path=save_path)
writer.close()

Epoch 1/200, Train Loss: 0.7342, Test Loss: 0.3816, Test MAER: 38.6843 %, Test MAE: 350.2345 MN, Test MSE: 235196.9824 MN^2 
Epoch 2/200, Train Loss: 0.2274, Test Loss: 0.1504, Test MAER: 19.2530 %, Test MAE: 187.2541 MN, Test MSE: 92687.2359 MN^2 
Epoch 3/200, Train Loss: 0.1160, Test Loss: 0.0987, Test MAER: 14.6019 %, Test MAE: 142.2609 MN, Test MSE: 60831.1663 MN^2 
Epoch 4/200, Train Loss: 0.0848, Test Loss: 0.0786, Test MAER: 12.6864 %, Test MAE: 121.2348 MN, Test MSE: 48422.6375 MN^2 
Epoch 5/200, Train Loss: 0.0696, Test Loss: 0.0661, Test MAER: 11.3707 %, Test MAE: 107.0768 MN, Test MSE: 40753.6490 MN^2 
Epoch 6/200, Train Loss: 0.0592, Test Loss: 0.0571, Test MAER: 10.3114 %, Test MAE: 96.2459 MN, Test MSE: 35176.5056 MN^2 
Epoch 7/200, Train Loss: 0.0514, Test Loss: 0.0500, Test MAER: 9.3860 %, Test MAE: 87.4846 MN, Test MSE: 30813.6794 MN^2 
Epoch 8/200, Train Loss: 0.0454, Test Loss: 0.0443, Test MAER: 8.5358 %, Test MAE: 79.8954 MN, Test MSE: 27333.4310 MN^2 
Epoch 9/200,

In [24]:
writer = SummaryWriter(f"./runs/final/cantilever/MLP/3/70/ReLU/lr_0.0004")
save_path = "./runs/final/cantilever/MLP/3/70/ReLU/lr_0.0004/SAVE"
os.makedirs(save_path, exist_ok=True)
model_beam = main_final(writer, _train_ds['cantilever'], _test_ds['cantilever'], hidden_size=40, n_hidden=4, activation=nn.ReLU, activation_params={}, learning_rate=0.0004, n_epochs=200, save_path=save_path)
writer.close()

Epoch 1/200, Train Loss: 0.9220, Test Loss: 0.7524, Test MAER: 99.0205 %, Test MAE: 584.2581 MN, Test MSE: 503149.2752 MN^2 
Epoch 2/200, Train Loss: 0.4517, Test Loss: 0.2468, Test MAER: 41.2354 %, Test MAE: 292.9272 MN, Test MSE: 165069.2741 MN^2 
Epoch 3/200, Train Loss: 0.1844, Test Loss: 0.1475, Test MAER: 29.2349 %, Test MAE: 215.0888 MN, Test MSE: 98613.3052 MN^2 
Epoch 4/200, Train Loss: 0.1223, Test Loss: 0.1096, Test MAER: 23.5557 %, Test MAE: 175.8310 MN, Test MSE: 73261.5040 MN^2 
Epoch 5/200, Train Loss: 0.0937, Test Loss: 0.0863, Test MAER: 19.8718 %, Test MAE: 147.8399 MN, Test MSE: 57711.3072 MN^2 
Epoch 6/200, Train Loss: 0.0748, Test Loss: 0.0699, Test MAER: 17.0421 %, Test MAE: 126.3958 MN, Test MSE: 46735.8095 MN^2 
Epoch 7/200, Train Loss: 0.0622, Test Loss: 0.0597, Test MAER: 15.4051 %, Test MAE: 112.2202 MN, Test MSE: 39895.2523 MN^2 
Epoch 8/200, Train Loss: 0.0533, Test Loss: 0.0515, Test MAER: 13.9405 %, Test MAE: 100.3845 MN, Test MSE: 34472.8279 MN^2 
Epoch 